# BLEND043
- Blend042 + Blend of top subs: -2.271LB

In [1]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

def log_mean_absolute_error(y_true, y_pred):
    return np.log(mean_absolute_error(y_true, y_pred))
%matplotlib inline

In [2]:
sub1 = pd.read_csv('../submissions/blend_top_subs-1.csv')
sub2 = pd.read_csv('../submissions/BLEND042_sub_-2.02923CV.csv')

# Blend

In [29]:
final_sub = sub1.copy()

In [30]:
final_sub['scalar_coupling_constant'] = ((3*sub1['scalar_coupling_constant']) + sub2['scalar_coupling_constant']) / 4

# Validate Sub

In [31]:
sub_good = pd.read_csv('../submissions/BLEND040_sub_-2-Copy1.01768CV.csv')
final_sub['good_scc'] = sub_good['scalar_coupling_constant']
final_sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.00000,0.99999
good_scc,0.99999,1.00000


In [32]:
test = pd.read_csv('../input/test.csv')

In [33]:
final_sub = final_sub.merge(test, on='id')

In [34]:
print('MAE vs Good Sub')
for i, d in final_sub.groupby('type'):
    mae_t = mean_absolute_error(d['good_scc'], d['scalar_coupling_constant'])
    corr_t = np.corrcoef(d['good_scc'], d['scalar_coupling_constant'])[1][0]
    print(f'{i} - {mae_t:0.5f} - {corr_t:0.5f}')

MAE vs Good Sub
1JHC - 0.19038 - 0.99986
1JHN - 0.06375 - 0.99996
2JHC - 0.06615 - 0.99967
2JHH - 0.03467 - 0.99989
2JHN - 0.02874 - 0.99991
3JHC - 0.07965 - 0.99897
3JHH - 0.04491 - 0.99967
3JHN - 0.02435 - 0.99950


# Create Submission and OOF Files

In [35]:
import json
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_name():
    """
    Return the full path of the jupyter notebook.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [36]:
BLEND_NUMBER = get_notebook_name().split('/')[-1].replace('.ipynb','').replace('-','')
print(f'NAME TO SAVE {BLEND_NUMBER}')

NAME TO SAVE BLEND043


In [38]:
# Save the Results
final_sub[['id','scalar_coupling_constant']].to_csv(f'../submissions/{BLEND_NUMBER}_sub_BLEND042_plus_blendtopsubs.csv', index=False)
#final_oof.to_csv(f'../oof/{BLEND_NUMBER}_oof_{glmae:0.5f}CV.csv', index=False)